# note to self: conda env = fastml

In [1]:
import pandas as pd 
import numpy as np

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer, SimpleImputer

from tpot import TPOTClassifier

import os

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [4]:
rs = 1

In [5]:
def read_data(file):
    """
    
    """
    df = (pd.read_csv(file, dtype = {
                            "PassengerId":np.int32,
                            "Name": "object",
                            "Pclass":np.int,
                            "Survived":np.int32,
                            "Sex":"object",
                            "Age":np.float,
                            "SibSp":np.int,
                            "Embarked":"object",
                            "Cabin":"object",
                            "Fare":np.float64,
                        },)
            .rename(columns=str.lower)
            .set_index("passengerid")
            #.pipe(onehot_name)
            #.pipe(feature_egneenering)
            .rename(columns=str.lower)
            .drop(['cabin','name','ticket'],axis=1)
        )
    return df
    

In [6]:
filename = "../data/train.csv"
data = read_data(filename)
X_train = data.drop(['survived'],axis=1) 
y_train = data.survived

In [31]:
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = X_train.select_dtypes(exclude = 'O').columns.values #['age','fare','mr', 'miss', 'mrs', 'master']
numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(random_state=0,initial_strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = X_train.select_dtypes(include = 'O').columns.values#['Embarked', 'Sex', 'Pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent',missing_values=np.nan)),
    ('onehot', OneHotEncoder(handle_unknown='error', drop="first"))
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
        ])

In [32]:
X_train_processed = preprocessor.fit_transform(X_train)
np.shape(X_train_processed)

(891, 8)

In [9]:
magic = TPOTClassifier(n_jobs=-1,random_state=rs,verbosity=3)

In [10]:
from ipywidgets import IntProgress


In [11]:
magic.fit(X_train_processed,y_train,)

31 operators have been imported by TPOT.


_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only dual=False, got dual=True.
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=False.
_pre_test decorator: _random_mutation_operator: num_test=1 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=2 Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=True.
_pre_test decorator: _random_mutation_operator: num_test=0 Negative values in data passed to MultinomialNB (input X).
_pre_test decorator: _mate_operator: num_test=0 Negative values in data passed to MultinomialNB (input X).
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=100,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=100,
               random_state=1, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=3, warm_start=False)

In [12]:
magic

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=100,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=100,
               random_state=1, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=3, warm_start=False)

In [42]:
new_magic = magic.fitted_pipeline_

In [43]:
new_magic.fit(X_train_processed,y_train)

Pipeline(memory=None,
         steps=[('variancethreshold', VarianceThreshold(threshold=0.2)),
                ('polynomialfeatures',
                 PolynomialFeatures(degree=2, include_bias=False,
                                    interaction_only=False, order='C')),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=False, ccp_alpha=0.0,
                                        class_weight=None, criterion='entropy',
                                        max_depth=None,
                                        max_features=0.7500000000000001,
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=9, min_samples_split=3,
                                        min_weight_fraction_leaf=0.0,
                                        n

In [19]:
magic.export('tpot_exported_pipeline.py')


In [55]:
cv_innder = StratifiedKFold(shuffle=True, random_state = rs)

magic_truth = cross_val_score(new_magic,X_train_processed,y_train,cv=cv_innder,n_jobs=-1)

array([0.79888268, 0.86516854, 0.82022472, 0.82022472, 0.84831461])

In [44]:
new_magic.predict(X_test_proccessed)

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [46]:
def submit(model,filename):
    """
    
    """
    X_test = read_data("../data/test.csv")
    X_test_processed = preprocessor.transform(X_test)
    predictions = model.predict(X_test_processed)
    submission = pd.DataFrame({'PassengerId':X_test.index,'Survived':predictions})
    submission.to_csv(filename,index=False)
    print('Saved file: ' + filename)
    return

In [21]:
file = "tpot.csv"

In [47]:
submit(new_magic,file)


Saved file: tpot.csv


In [51]:
!kaggle competitions submit -c titanic -f $file -m "tpot"

100%|████████████████████████████████████████| 2.77k/2.77k [00:04<00:00, 663B/s]
Successfully submitted to Titanic: Machine Learning from Disaster

In [54]:
!kaggle competitions submissions titanic 

fileName                 date                 description                                   status    publicScore  privateScore  
-----------------------  -------------------  --------------------------------------------  --------  -----------  ------------  
tpot.csv                 2020-03-19 08:55:54  tpot                                          complete  0.73684      None          
ensamble_first.csv       2020-03-06 11:06:11  babies                                        complete  0.80382      None          
fist_svc.csv             2020-03-05 15:30:19  first svc                                     complete  0.79425      None          
two_more.csv             2020-03-05 14:24:54  fine tuned first rf_sfs with overall fitting  complete  0.79904      None          
one_more.csv             2020-03-05 14:22:57  fine tuned first rf_sfs                       complete  0.79904      None          
fine_tuned_rf_sfs.csv    2020-03-05 13:16:05  new features rfr with sfs fine tuned        